In [1]:
import os
import torch
import numpy as np
from scipy.io import wavfile
from shutil import copyfile
from tqdm import tqdm

from Inference import Inferencer

%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

In [2]:
import matplotlib as mpl
# 유니코드 깨짐현상 해결
mpl.rcParams['axes.unicode_minus'] = False
# 나눔고딕 폰트 적용
plt.rcParams["font.family"] = 'NanumGothic'

# Hyper parameters

In [3]:
os.environ['CUDA_VISIBLE_DEVICES']= '7' # Left space
checkpoint_Paths = {
    'EY': '/data/results/Tacotron2/GST.EMO_YUA/Checkpoint',
#     'Y_FOU': '/data/results/Tacotron2/GST.YUAFOU_FT/Checkpoint',
#     'Y_ALL': '/data/results/Tacotron2/GST.YUAALL_FT/Checkpoint',
    'AIHUB': '/data/results/Tacotron2/GST.AIHub/Checkpoint'
    }
checkpoint_Paths = {
    key: max([
        os.path.join(root, file).replace('\\', '/')                
        for root, _, files in os.walk(path)
        for file in files
        if os.path.splitext(file)[1] == '.pt'
        ], key = os.path.getctime
        )
    for key, path in checkpoint_Paths.items()
    }

hp_Paths = {
    key: os.path.join(os.path.dirname(path), 'Hyper_Parameter.yaml')
    for key, path in checkpoint_Paths.items()
    }

out_Paths = {
    key: './{}_Result_{}K'.format(key, os.path.splitext(os.path.basename(value))[0].split('_')[1][:-3])
    for key, value in checkpoint_Paths.items()
    }

# ref_Sources_Path = {
#     os.path.splitext(file)[0]: os.path.join(root, file)
#     for root, _, files in os.walk('./FOU_Filtered_Wav')
#     for file in files
#     if os.path.splitext(file)[1].lower() == '.wav'
#     }
# ref_Sources_Path['Neutral']= './Wav_for_Inference/YUA_NEUTRAL.wav'
ref_Sources_Path = {}
ref_Sources_Path.update({
    os.path.splitext(file)[0]: os.path.join(root, file)
    for root, _, files in os.walk('./AIHub_Emotion_Wav')
    for file in files
    if os.path.splitext(file)[1].lower() == '.wav'
    })
ref_Sources_Path.update({
    os.path.splitext(file)[0]: os.path.join(root, file)
    for root, _, files in os.walk('./JPS_Wav')
    for file in files
    if os.path.splitext(file)[1].lower() == '.wav'
    })
ref_Sources_Path.update({
    os.path.splitext(file)[0]: os.path.join(root, file)
    for root, _, files in os.walk('./YUA_Wav')
    for file in files
    if os.path.splitext(file)[1].lower() == '.wav'
    })

batch_Size = 16

# Model load

In [4]:
inferencer_Dict = {
    key: Inferencer(hp_path= hp_Paths[key], checkpoint_path= checkpoint_Path, out_path= out_Paths[key], batch_size= batch_Size)
    for key, checkpoint_Path in checkpoint_Paths.items()
    }
for inferencer in inferencer_Dict.values():
    inferencer.model.hp_Dict['Ignore_Stop'] = False

In [5]:
vocoder = torch.jit.load('vocoder.pts').to(list(inferencer_Dict.values())[0].device)

# Insert list

In [6]:
# texts = [    
#     '응! 완전 여신포스! 저 아닌 거 같아요!',
#     '어제 선배 번호 물어보는걸 깜박해서요.',
#     '진짜에요. 저 의상학과잖아요.',
#     '그럼 어디서 찍을까요?',
#     '선배와 나의 첫 작품!',
#     '네! 인정! 진짜 맛있어요!',
#     '그럼 이만 일하러 가실까요, 작가님?',
#     '사진 찍었어요? 어때요?',
#     '무더운 여름! 스마일 소다와 함께 하세요!',
#     '뭐 입을 지 몰라서 일단 다 가지고 왔죠!',
#     '여기 나무쪽에 서볼까? 이렇게?',
#     ]
texts = [
    '안녕하세요! 여기는 스마일게이트 에이아이 센터입니다!',
    '자세한 정보는 에이아이쩜, 스마일게이트쩜, 넷으로 접속해서 확인하세요!',
#     '선배! 제 목소리는 언제 완성되는거죠?',
#     '선배! 또 토마토 넣었죠? 토마토는 싫어요!.',
#     '세아는 조금 소란스럽긴 하지만 보고있으면 재미있는 친구에요!',
#     '선배? 다음주에 시간 어때요? 저 영화보고 싶어요',
#     '이번주엔 게임데이터랑 직접 녹음한 데이터랑 같이 써서 다시 말하는 법을 배울꺼에요!',
#     '스마일게이트 메가포트가 직접 개발한 신작! 마법양품점! 지금 바로 시작해보세요!',
#     '선배, 어떤 옷이 더 사진찍기 좋아보여요? 다 어울린다고요? 아이 참!.',
#     '포커스 온 유는 스마일게이트 귀여운 미소녀인 저 한유아가 여자주인공으로 나오는 브이알게임이에요.',
#     '전 유튜브 방송과 코스프레가 취미에요.',
#     '내가 왜 화났는지 몰라요? 됐어요! 선배는 항상 이런식이야!'
    ]

In [7]:
for path in out_Paths.values():
    os.makedirs(path, exist_ok= True)
    
refs, ref_paths = zip(*ref_Sources_Path.items())

for inferencer_Label, inferencer in inferencer_Dict.items():
    print('Inferencer: {}'.format(inferencer_Label))
    for index, text in tqdm(enumerate(texts)):
        mels, stops = inferencer.Inference_Epoch(
            texts= [text] * len(ref_paths),
            speaker_labels= refs,
            speakers= ref_paths,
            reference_labels= refs,
            references= ref_paths,
            use_tqdm= False
            )
        
        mels = [
            mel[:,:(stop <= 0.0).nonzero()[0]] if torch.any(stop <= 0.0).cpu().numpy() else mel
            for mel, stop in zip(mels, stops)
            ]

        mels = [
            torch.nn.functional.pad(mel[None,], (2,2), 'reflect')
            for mel in mels
            ]

        max_length = max([mel.size(2) for mel in mels])
        mels = torch.cat([
            torch.nn.functional.pad(mel, (0,max_length - mel.size(2)), value=-4.0)
            for mel in mels
            ], dim= 0)

        x = torch.randn(size=(mels.size(0), 256 * (mels.size(2) - 4))).to(mels.device)
        wavs = vocoder(x, mels).cpu().numpy()
        wavs = [
            wav[:(stop <= 0.0).nonzero()[0].cpu().numpy()[0] * 256] if torch.any(stop <= 0.0).cpu().numpy() else wav
            for wav, stop in zip(wavs, stops)
            ]

        for wav, ref in zip(wavs, refs):
            wavfile.write(
                os.path.join(out_Paths[inferencer_Label], 'TTS.IDX_{:03d}.REF_{}.wav'.format(index, ref)),
                24000,
                (wav * 32767.5).astype(np.int16))

0it [00:00, ?it/s]

Inferencer: EY


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/python_arg_parser.cpp:882.)
2it [00:35, 17.97s/it]
0it [00:00, ?it/s]

Inferencer: AIHUB


2it [00:24, 12.07s/it]
